In [ ]:
!pip install transformers
!pip gdown

ERROR: unknown command "gdown" - maybe you meant "download"


In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1ecTFXIaUwAh7I7iLeDK8YCMcPoU4wVwx'
output = 'kogpt_chat_model.pt'  # 원하는 파일 이름과 확장자로 변경
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ecTFXIaUwAh7I7iLeDK8YCMcPoU4wVwx
From (redirected): https://drive.google.com/uc?id=1ecTFXIaUwAh7I7iLeDK8YCMcPoU4wVwx&confirm=t&uuid=212d7761-5b9f-48f7-ad46-6d6bff68f904
To: /content/kogpt_chat_model.pt
100%|██████████| 501M/501M [00:04<00:00, 113MB/s]


'kogpt_chat_model.pt'

In [ ]:
# 라이브러리 임포트
import torch
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast

# Q(질의) > lm > A(응답)
# Q(질의-사용자) > lm > A(응답-시스템)
# <usr> > lm > <sys>
# "<usr>사용자질문 ... <sys>" > lm > "시스템 답변"

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)

# Pretrained 모델의 구성을 로드합니다.
config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')
model = GPT2LMHeadModel(config)
model.load_state_dict(torch.load('kogpt_chat_model.pt'))
model.eval()

sent = "0"

def dl_model(input_text, max_length=40):
    response = ""
    for _ in range(max_length):
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + input_text + SENT + sent + A_TKN + response)).unsqueeze(dim=0)
        attention_mask = input_ids != koGPT2_TOKENIZER.pad_token_id
        pred = model(input_ids, attention_mask=attention_mask)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        response += gen.replace("▁", " ")

    return response.strip()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = dl_model(user_message)
    chat_history.append({'user': user_message, 'ai': ai_message})
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > 안녕
 A I > 안녕하세요
USER > 저는 김태영입니다. 
 A I > 저를 만들어 준 사람을 부모님  저랑 이야기해 주는 사람을 친구로 생각하고 있어요
USER > 점심 메뉴를 추천해줘
 A I > 점심식사 후가 좋죠
USER > 커피 한 잔 해볼까?
 A I > 커피가 정말 좋아요
USER > 감사합니다. 
 A I > 저도 듣고 싶네요
USER > quit
